In [ ]:
sc

Out[1]:

SparkContext 

 Spark UI 

 
 Version 
 v3.1.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [ ]:
from pyspark import SparkFiles
import pandas as pd
from io import StringIO
import boto3
from botocore.client import Config

In [ ]:
def get_page_rank(input_file_url, max_iter, output_file_name):
  spark.sparkContext.addFile(url)
#   loading the data
  data = spark.read.csv("file://"+SparkFiles.get('flights_data.csv'), header= True, inferSchema= True).rdd
# getting only the origin and destination code
  origin_destination = data.map(lambda x: (x[0],x[2]))
#   outlinks from the origin to destination
  outLinks = origin_destination.groupByKey().map(lambda x: (x[0],len(list(x[1]))))
# all outgoing flights from origin 
  origin_all_outgoing_flights = origin_destination.groupByKey().map(lambda x: (x[0],list(x[1])))
# getting all unique airport keys using join of origin airport keys and destination airport keys
  origin_airport_keys = origin_destination.keys().distinct().map(lambda x: (x,1))
  destination_airport_keys = origin_destination.values().distinct().map(lambda x: (x,1))
#   unique airport keys
  airport_keys = origin_airport_keys.fullOuterJoin(destination_airport_keys).map(lambda x: x[0])
# initializing pageRank to 10
  pageRank_airports = airport_keys.map(lambda x: (x,10))
# initializing the damping factor
  alpha = 0.15
#   calculating total airports
  total_airports = origin_airport_keys.fullOuterJoin(destination_airport_keys).map(lambda x: (1,1)).reduceByKey(lambda x,y: x+y).collect()[0][1]

#   iterating steps for pagerank calculations
  for i in range(0,int(max_iter)):
#     generate intermediate output by joining
    intermediate1 = origin_destination.join(pageRank_airports)
#   getting the sum of numerator and denominators i.e Summation of PR(Ti)/C(Ti)
    intermediate2 = intermediate1.join(outLinks).map(lambda x: (x[1][0][0],(x[1][0][1]/x[1][1]))).reduceByKey(lambda x,y : x+y)
#     get the new pageRank using the formula
    pageRank_airports = intermediate2.map(lambda x: (x[0],  ((alpha*(1/total_airports)) + ((1-alpha)*x[1]))))

#   sorting the output of page rank by descending order
  output_page_rank = pageRank_airports.sortBy(lambda x: -x[1]).collect()
# writing the output to the aws file
  write_output_to_aws(output_file_name, output_page_rank)
# displaying output for safer side
  display(output_page_rank)
  
  
  
def write_output_to_aws(output_file_name, output_page_rank):
  # could have written the below configuration in the config file and loaded the keys
  ACCESS_KEY = ''
  SECRET_KEY = ''
  AWS_BUCKET_NAME = ""
  s3 = boto3.resource('s3', aws_access_key_id = ACCESS_KEY, aws_secret_access_key =SECRET_KEY, config = Config(signature_version = 's3v4') )
#   loading the output to pandas dataframe so as to store output in a single .csv file instead of part csv files
  op = pd.DataFrame(output_page_rank)
  op.columns = ['AirportCode','PageRank']
  csv_buffer = StringIO()
  op.to_csv(csv_buffer)
  file_name = f'output_new/{output_file_name}.csv'
  s3.Object(AWS_BUCKET_NAME, file_name).put(Body=csv_buffer.getvalue(),ACL='public-read')
  region= "us-west-1"
  url = f"https://{AWS_BUCKET_NAME}.s3.{region}.amazonaws.com/{file_name}"
  print("--------------------------------")
  print("download url from aws: ", url)
  print("--------------------------------")
  

if __name__=="__main__":
  url = "https://myawsbucketforbigdata.s3.amazonaws.com/flights_data.csv"
  max_iter = 10
  output_file_name = "output_page_rank"
  get_page_rank(url, max_iter, output_file_name)

--------------------------------
download url from aws: https://bigdataproject.s3.us-west-1.amazonaws.com/output_new/output_page_rank.csv
--------------------------------

_1,_2
DEN,43.70812475060697
ORD,42.55456052472752
ATL,32.5093015960192
DFW,30.374414652704008
ANC,27.737720448244673
PHX,26.808228514054562
LAX,25.75583427106883
BET,25.27608614983726
MCO,24.789771423930365
LAS,23.672890403869328
